In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

C:\Users\mxd162430\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.__version__

'1.6.0'

### Configuration of Neural Network
The configuration of the Convolutional Neural Network is defined for convenience. These numbers can be changed for experimental puprose.

In [3]:
# Convolutional Layer 1.
filter_size1 = 5          # Convolution filters are 5 x 5 pixels.
num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 5          # Convolution filters are 5 x 5 pixels.
num_filters2 = 36         # There are 36 of these filters.

# Fully-connected layer.
fc_size = 128             # Number of neurons in fully-connected layer.

### Loading Data
The MNIST dataset is about 12MB and will be downloaded automatically if not already present in the given path.

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot = True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


The MNIST dataset is now loaded and consists of 70K images with associated class labels. The dataset is split into 3 mutually exclusive subsets. We only use the training and test sets in this tutorial.

In [5]:
print("Size of:")
print("\b Training-set:\t\t{}".format(len(data.train.labels)))
print("\b Test-set:\t\t{}".format(len(data.test.labels)))
print("\b Validation-set:\t{}".format(len(data.validation.labels)))

Size of:
 Training-set:		55000
 Test-set:		10000
 Validation-set:	5000


The class-labels are One-Hot encoded, that is, each label is a vector with 10 elements, all of which are zero except for one element. The index of this one element is the class-number, that is, the digit shown in the associated image. We also need the class-numbers as integers for the test-set, so we calculate it now.

In [6]:
data.test.cls = np.argmax(data.test.labels, axis = 1)

### Data Dimensions
The data dimensions are used in several places in this source-code. They are defined here so as to make it easy to use them throughout the code as and when needed.

In [7]:
# We know that MNIST images are 28 pixels in each dimension.
img_size = 28

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = 1

# Number of classes, one class for each of 10 digits.
num_classes = 10